In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [18]:
import kagglehub

import os

path = kagglehub.dataset_download("rabieelkharoua/alzheimers-disease-dataset")

In [19]:
files = os.listdir(path)
print("Content of", files)

csv_file = files[0]
csv_path = os.path.join(path, csv_file)

# Load DataFrame
df = pd.read_csv(csv_path)
df = df.drop(columns=["DoctorInCharge"])  # Drop useless column


# Display the content of DataFrame
df.head().T

Content of ['alzheimers_disease_data.csv']


,0,1,2,3,4
PatientID,4751.000000,4752.000000,4753.000000,4754.000000,4755.000000
Age,73.000000,89.000000,73.000000,74.000000,89.000000
Gender,0.000000,0.000000,0.000000,1.000000,0.000000
Ethnicity,0.000000,0.000000,3.000000,0.000000,0.000000
EducationLevel,2.000000,0.000000,1.000000,1.000000,0.000000
BMI,22.927749,26.827681,17.795882,33.800817,20.716974
Smoking,0.000000,0.000000,0.000000,1.000000,0.000000
AlcoholConsumption,13.297218,4.542524,19.555085,12.209266,18.454356
PhysicalActivity,6.327112,7.619885,7.844988,8.428001,6.310461
DietQuality,1.347214,0.518767,1.826335,7.435604,0.795498


<!-- @format -->

# First evaluation


In [20]:
from sklearn.model_selection import train_test_split
from functions.data_prep import data_preprocessing

X = np.array(df.drop(columns=["Diagnosis"]))
y = np.array(df["Diagnosis"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train_scaled, X_test_scaled = data_preprocessing(X_train, X_test)

<!-- @format -->

## Logisitc Regression


In [21]:
from models.logistic_regression import Logistic_Regression_

clf = Logistic_Regression_()
print(clf.bayesian_opt(X_train_scaled, y_train))

[I 2025-01-28 00:00:27,561] A new study created in memory with name: no-name-59cc6a3b-fca7-4760-8779-eecfd4e87295
[I 2025-01-28 00:00:27,638] Trial 0 finished with value: 0.8359329446064139 and parameters: {'solver': 'saga', 'penalty': 'l2', 'C': 10}. Best is trial 0 with value: 0.8359329446064139.
[I 2025-01-28 00:00:27,752] Trial 1 finished with value: 0.8359329446064139 and parameters: {'solver': 'sag', 'penalty': 'l1', 'C': 10}. Best is trial 0 with value: 0.8359329446064139.
[I 2025-01-28 00:00:27,785] Trial 2 finished with value: 0.8359329446064139 and parameters: {'solver': 'lbfgs', 'penalty': None, 'C': 100}. Best is trial 0 with value: 0.8359329446064139.
[I 2025-01-28 00:00:27,816] Trial 3 finished with value: 0.8353515492575768 and parameters: {'solver': 'liblinear', 'penalty': None, 'C': 1}. Best is trial 0 with value: 0.8359329446064139.
[I 2025-01-28 00:00:27,881] Trial 4 finished with value: 0.8359329446064139 and parameters: {'solver': 'saga', 'penalty': None, 'C': 100}

0.8405925825479693


<!-- @format -->

Do zrobienia

- Dodanie opcji sprawdzenia wszystkich wyników


<!-- @format -->


<!-- @format -->

## Suppor Vector Machines


In [22]:
from models.svm import SVM

svm_model = SVM()
print(svm_model.bayesian_opt(X_train_scaled, y_train))

[I 2025-01-28 00:00:32,099] A new study created in memory with name: no-name-47f25148-9d85-407d-89eb-9ec690c9e1e9
[I 2025-01-28 00:00:32,363] Trial 0 finished with value: 0.8336242001163466 and parameters: {'C': 1.6019280996235596, 'gamma': 'scale', 'kernel': 'rbf', 'degree': 4, 'coef0': 0.6136843602276734}. Best is trial 0 with value: 0.8336242001163466.
[I 2025-01-28 00:00:32,486] Trial 1 finished with value: 0.7108784176847004 and parameters: {'C': 0.03520769013750258, 'gamma': 'scale', 'kernel': 'poly', 'degree': 4, 'coef0': 0.578434815427669}. Best is trial 0 with value: 0.8336242001163466.
[I 2025-01-28 00:00:44,430] Trial 2 finished with value: 0.840023269342641 and parameters: {'C': 78.15665773315965, 'gamma': 'auto', 'kernel': 'linear', 'degree': 4, 'coef0': 0.9195030329844103}. Best is trial 2 with value: 0.840023269342641.
[I 2025-01-28 00:00:44,533] Trial 3 finished with value: 0.726585223967423 and parameters: {'C': 0.001414491878843855, 'gamma': 'auto', 'kernel': 'linear'

0.8452588714368819


<!-- @format -->

## XGBoost


In [23]:
# from models.xgboost import XGBoost

# xgboost_model = XGBoost()
# print(xgboost_model.bayesian_opt(X_train_scaled, y_train))

<!-- @format -->

## Random Forest


In [24]:
from models.random_forest import Random_Forest_

random_forest_model = Random_Forest_()
print(random_forest_model.bayesian_opt(X_train_scaled, y_train))

[I 2025-01-28 00:01:02,872] A new study created in memory with name: no-name-75bfcbe1-256c-47f4-95de-19a3fb934d42
[I 2025-01-28 00:01:06,178] Trial 0 finished with value: 0.9016870273414775 and parameters: {'n_estimators': 390, 'max_depth': 14, 'min_samples_split': 18, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 0.9016870273414775.
[I 2025-01-28 00:01:09,150] Trial 1 finished with value: 0.9139034322280396 and parameters: {'n_estimators': 323, 'max_depth': 13, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9139034322280396.
[I 2025-01-28 00:01:12,104] Trial 2 finished with value: 0.9296102385107621 and parameters: {'n_estimators': 126, 'max_depth': 28, 'min_samples_split': 18, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 2 with value: 0.9296102385107621.
[I 2025-01-28 00:01:13,734] Trial 3 finished with value: 0.9133216986620128 and parameters: {'n_estimators': 185, 'max_depth': 20, 'm

0.9447353112274578
